In [1]:
sc


org.apache.spark.SparkContext@d8fce0d4

In [2]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

In [4]:
val df=sqlContext.read.parquet("MGH-7P-CENTERS/7PKMeansModel/data")

In [5]:
df.printSchema()

root
 |-- clusterIdx: integer (nullable = false)
 |-- clusterCenter: struct (nullable = true)
 |    |-- type: byte (nullable = false)
 |    |-- size: integer (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = false)



In [7]:
df.head()

[0,[1,null,null,WrappedArray(-0.9464129761746461, -0.0016218862947692985, 0.05027471251675777, 0.041531146654619495, 0.3001746656314373, -0.054843468802641605, 0.07667365329030061, -0.019320479067146838, -0.009743639023073071, -3.7164263904923615E-4, 0.005564500150812339, -0.013398909066944516, 0.009191256978025645, -4.1980627268278293E-4, 2.5568412365770213E-4, 0.0030034975847473464, 0.0047179961303550145, 0.0033703462576947446, -8.964255285432724E-4, 9.394639596460496E-4, -0.002550473406322839, -0.0018348127334704557, 5.18325587390249E-4, -1.6953583842543935E-4, -0.0011923145801090592, -3.31626419863201E-4, -1.829016451072681E-4, -9.313124200842011E-5, -4.910190205287119E-4, 1.480870789173435E-4, -1.4283207759178519E-5, 3.3336968417905225E-4, -9.648806...

In [8]:
df.count()

24762

In [10]:
val data1 = df.select("clusterCenter.values")
data1.getClass().getName()

org.apache.spark.sql.DataFrame

In [12]:
data1.printSchema()

root
 |-- values: array (nullable = true)
 |    |-- element: double (containsNull = false)



In [11]:
val data1RDD=data1.rdd
data1RDD.take(2)

Array([WrappedArray(-0.9464129761746461, -0.0016218862947692985, 0.05027471251675777, 0.041531146654619495, 0.3001746656314373, -0.054843468802641605, 0.07667365329030061, -0.019320479067146838, -0.009743639023073071, -3.7164263904923615E-4, 0.005564500150812339, -0.013398909066944516, 0.009191256978025645, -4.1980627268278293E-4, 2.5568412365770213E-4, 0.0030034975847473464, 0.0047179961303550145, 0.0033703462576947446, -8.964255285432724E-4, 9.394639596460496E-4, -0.002550473406322839, -0.0018348127334704557, 5.18325587390249E-4, -1.6953583842543935E-4, -0.0011923145801090592, -3.31626419863201E-4, -1.829016451072681E-4, -9.313124200842011E-5, -4.910190205287119E-4, 1.480870789173435E-4, -1.4283207759178519E-5, 3.3336968417905225E-4, -9.64880682...

In [13]:
val data1Double = data1.map(r=>r.getList(0))
data1Double.take(2)

Array([-0.9464129761746461, -0.0016218862947692985, 0.05027471251675777, 0.041531146654619495, 0.3001746656314373, -0.054843468802641605, 0.07667365329030061, -0.019320479067146838, -0.009743639023073071, -3.7164263904923615E-4, 0.005564500150812339, -0.013398909066944516, 0.009191256978025645, -4.1980627268278293E-4, 2.5568412365770213E-4, 0.0030034975847473464, 0.0047179961303550145, 0.0033703462576947446, -8.964255285432724E-4, 9.394639596460496E-4, -0.002550473406322839, -0.0018348127334704557, 5.18325587390249E-4, -1.6953583842543935E-4, -0.0011923145801090592, -3.31626419863201E-4, -1.829016451072681E-4, -9.313124200842011E-5, -4.910190205287119E-4, 1.480870789173435E-4, -1.4283207759178519E-5, 3.3336968417905225E-4, -9.648806828675189E-5, 2...

In [14]:
import org.apache.spark.sql.functions.explode
val data2 = data1.select(explode(data1("values")).alias("b_columns"))

In [15]:
data2.show()

+--------------------+
|           b_columns|
+--------------------+
| -0.9464129761746461|
|-0.00162188629476...|
| 0.05027471251675777|
|0.041531146654619495|
|  0.3001746656314373|
|-0.05484346880264...|
| 0.07667365329030061|
|-0.01932047906714...|
|-0.00974363902307...|
|-3.71642639049236...|
|0.005564500150812339|
|-0.01339890906694...|
|0.009191256978025645|
|-4.19806272682782...|
|2.556841236577021...|
|0.003003497584747...|
|0.004717996130355...|
|0.003370346257694...|
|-8.96425528543272...|
|9.394639596460496E-4|
+--------------------+
only showing top 20 rows



In [16]:
val data3=data2.rdd
data3.take(5)

Array([-0.9464129761746461], [-0.0016218862947692985], [0.05027471251675777], [0.041531146654619495], [0.3001746656314373])

In [17]:
data3.count()

2476200

In [18]:
%AddJar  https://github.com/kartha01/spark-tsne/raw/master/bin/spark-tsne-core_2.10-0.1-SNAPSHOT.jar

Using cached version of spark-tsne-core_2.10-0.1-SNAPSHOT.jar


In [19]:
import com.github.saurfang.spark.tsne.impl._
import com.github.saurfang.spark.tsne.tree.SPTree
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

In [20]:
val data3doubVals = data3.map{ row =>   row.getDouble(0) }
data3doubVals.getClass().getName()
data3doubVals.take(2)

Array(-0.9464129761746461, -0.0016218862947692985)

In [21]:
val mean = data3doubVals.mean()
val std = data3doubVals.stdev()

In [22]:
mean

-0.0029336185023481594

In [23]:
std

0.09993674188503057

In [24]:
data1Double.take(2).foreach(println)

[-0.9464129761746461, -0.0016218862947692985, 0.05027471251675777, 0.041531146654619495, 0.3001746656314373, -0.054843468802641605, 0.07667365329030061, -0.019320479067146838, -0.009743639023073071, -3.7164263904923615E-4, 0.005564500150812339, -0.013398909066944516, 0.009191256978025645, -4.1980627268278293E-4, 2.5568412365770213E-4, 0.0030034975847473464, 0.0047179961303550145, 0.0033703462576947446, -8.964255285432724E-4, 9.394639596460496E-4, -0.002550473406322839, -0.0018348127334704557, 5.18325587390249E-4, -1.6953583842543935E-4, -0.0011923145801090592, -3.31626419863201E-4, -1.829016451072681E-4, -9.313124200842011E-5, -4.910190205287119E-4, 1.480870789173435E-4, -1.4283207759178519E-5, 3.3336968417905225E-4, -9.648806828675189E-5, 2.447355045671867E-4, 3.448579209801694E-4, 4.2196921890244475E-5, -5.381365710719391E-5, 2.7781548746567412E-5, 1.6480931539516395E-4, 6.21936932494263E-5, 1.2770214487456201E-6, 1.9111572755521224E-5, -2.30522309745722E-5, -9.367002422469221E-5, 3.

In [25]:
val data1DoubleWithIdx=data1Double.zipWithIndex()
data1DoubleWithIdx.take(2).foreach(println)

([-0.9464129761746461, -0.0016218862947692985, 0.05027471251675777, 0.041531146654619495, 0.3001746656314373, -0.054843468802641605, 0.07667365329030061, -0.019320479067146838, -0.009743639023073071, -3.7164263904923615E-4, 0.005564500150812339, -0.013398909066944516, 0.009191256978025645, -4.1980627268278293E-4, 2.5568412365770213E-4, 0.0030034975847473464, 0.0047179961303550145, 0.0033703462576947446, -8.964255285432724E-4, 9.394639596460496E-4, -0.002550473406322839, -0.0018348127334704557, 5.18325587390249E-4, -1.6953583842543935E-4, -0.0011923145801090592, -3.31626419863201E-4, -1.829016451072681E-4, -9.313124200842011E-5, -4.910190205287119E-4, 1.480870789173435E-4, -1.4283207759178519E-5, 3.3336968417905225E-4, -9.648806828675189E-5, 2.447355045671867E-4, 3.448579209801694E-4, 4.2196921890244475E-5, -5.381365710719391E-5, 2.7781548746567412E-5, 1.6480931539516395E-4, 6.21936932494263E-5, 1.2770214487456201E-6, 1.9111572755521224E-5, -2.30522309745722E-5, -9.367002422469221E-5, 3

In [26]:
val scaledData = data1DoubleWithIdx.map(x => Vectors.dense(x._1.toArray().map(v => (v.asInstanceOf[Double] - mean) / std))).cache()
scaledData.take(2).foreach(println)

[-9.440765627097363,0.013125625098804067,0.5324201091158042,0.4449291053346616,3.033001460889008,-0.5194270827841444,0.7965766172788655,-0.16397233145393597,-0.06814331138150681,0.025635975467824205,0.08503497805578775,-0.10471914900563657,0.12132550302993195,0.025154034264568293,0.03191321395763438,0.05940874172109487,0.07656457963684428,0.06307955053502869,0.020384824793953347,0.03875534051780363,0.0038338761980663645,0.01099501292669511,0.03454129106699917,0.02765832277284546,0.01742406135515539,0.026036390954973773,0.02752458010293528,0.028422852364023417,0.024441456022545417,0.030836562440776434,0.02921183180003455,0.032690561298122235,0.028389262853148696,0.031803658463989096,0.032805516384554145,0.0297769906053356,0.02881627708609869,0.029632745627244748,0.031003890654228282,0.029977084894803097,0.029367532585495668,0.029545990987984844,0.0291240860615805,0.02841746113146861,0.029680324362511553,0.026440964697078552,0.026721951806049703,0.02564576695875535,0.029251118799955813,0

In [27]:
scaledData.count()

24762

In [28]:
val labels = data1DoubleWithIdx.map(_._2).collect()
val matrix = new RowMatrix(scaledData)
val pcaMatrix = matrix.multiply(matrix.computePrincipalComponents(50))
pcaMatrix.rows.cache()

MapPartitionsRDD[41] at mapPartitions at RowMatrix.scala:429

In [29]:
labels

Array(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, ...

In [30]:
import java.io.{BufferedWriter, OutputStreamWriter}

import org.apache.hadoop.fs.{FileSystem, Path}

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.SparkContext._
import org.slf4j.LoggerFactory

In [32]:
val hadoopConf = sc.hadoopConfiguration
val fs = FileSystem.get(hadoopConf)
   val costWriter = new BufferedWriter(new OutputStreamWriter(fs.create(new Path(s"MGH-7P-CENTERS/7Pt-SNE/cost.txt"), true)))

    //SimpleTSNE.tsne(pcaMatrix, perplexity = 20, maxIterations = 200)
    BHTSNE.tsne(pcaMatrix, maxIterations = 500, callback = {
    //LBFGSTSNE.tsne(pcaMatrix, perplexity = 10, maxNumIterations = 500, numCorrections = 10, convergenceTol = 1e-8)
      case (i, y, loss) =>
        if(loss.isDefined) println(s"$i iteration finished with loss $loss")

        val os = fs.create(new Path(s"MGH-7P-CENTERS/7Pt-SNE/result${"%05d".format(i)}.csv"), true)
        val writer = new BufferedWriter(new OutputStreamWriter(os))
        try {
          (0 until y.rows).foreach {
            row =>
              writer.write(labels(row).toString)
              writer.write(y(row, ::).inner.toArray.mkString(",", ",", "\n"))
              //writer.write(y(row, ::).inner.toArray.mkString(, ",", "\n"))
          }
          //if(loss.isDefined) costWriter.write(loss.get + "\n")
          try {
                costWriter.write(loss.get + "\n")
          } catch {
                case e: Exception => costWriter.write("No losses ("+None+") \n")
          }
          //costWriter.write(loss.get + "\n")  
        } finally {
          writer.close()
        }
    })
    costWriter.close()

10 iteration finished with loss Some(0.001306208470011048)
20 iteration finished with loss Some(0.0013062084700350904)
30 iteration finished with loss Some(0.0013062084682857794)
40 iteration finished with loss Some(0.001306208221270347)
50 iteration finished with loss Some(0.0013061007098138881)
60 iteration finished with loss Some(0.0012787416785375422)
70 iteration finished with loss Some(0.0011408004247147482)
80 iteration finished with loss Some(0.0010790390603662625)
90 iteration finished with loss Some(0.0010393388246308442)
100 iteration finished with loss Some(9.958751304881578E-4)
110 iteration finished with loss Some(1.793604791550502E-4)
120 iteration finished with loss Some(1.6772333458448606E-4)
130 iteration finished with loss Some(1.590268827053706E-4)
140 iteration finished with loss Some(1.5165684744846637E-4)
150 iteration finished with loss Some(1.452469675776332E-4)
160 iteration finished with loss Some(1.396367762655109E-4)
170 iteration finished with loss Some(1.